In [66]:
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import *

In [67]:
engine = create_engine('sqlite:///demo.db')
Base = declarative_base()

In [68]:
class Users(Base):
    __tablename__ = "users"
    UserId = Column(Integer, primary_key=True)
    Title = Column(String)
    FirstName = Column(String)
    LastName = Column(String)
    Email = Column(String)
    Username = Column(String)
    DOB = Column(DateTime)

class Uploads(Base):
    __tablename__ = "uploads"
    UploadId = Column(Integer, primary_key=True)
    UserId = Column(Integer)
    Title = Column(String)
    Body = Column(String)
    Timestamp = Column(DateTime)

In [69]:
Users.__table__.create(bind=engine, checkfirst=True)
Uploads.__table__.create(bind=engine, checkfirst=True)

In [70]:
import requests
url = 'https://randomuser.me/api/?results=10'
users_json = requests.get(url).json()
url2 = 'https://jsonplaceholder.typicode.com/posts/'
uploads_json = requests.get(url2).json()

In [12]:
#users_json
#uploads_json

{'results': [{'gender': 'female',
   'name': {'title': 'mademoiselle', 'first': 'cornelia', 'last': 'moulin'},
   'location': {'street': '5411 rue du moulin',
    'city': 'vordemwald',
    'state': 'vaud',
    'postcode': 9923,
    'coordinates': {'latitude': '-53.4197', 'longitude': '21.7156'},
    'timezone': {'offset': '+5:45', 'description': 'Kathmandu'}},
   'email': 'cornelia.moulin@example.com',
   'login': {'uuid': '3b1c8dee-c2d8-4bdf-9fa6-a5d1e5f8fa21',
    'username': 'happyostrich396',
    'password': 'pavlov',
    'salt': 'i1ITY9h9',
    'md5': '729eba35f3285dc4a5674bb52a859dc8',
    'sha1': '03a8c86e04a23a5f37e412835d9c86400553c117',
    'sha256': '6c463305a8049c16b7fbcb838ab1375cd0c70b85482a129aefd399822a2339f7'},
   'dob': {'date': '1990-03-24T11:18:04Z', 'age': 29},
   'registered': {'date': '2003-04-29T02:15:02Z', 'age': 16},
   'phone': '(596)-110-2084',
   'cell': '(203)-081-6860',
   'id': {'name': 'AVS', 'value': '756.9696.3467.23'},
   'picture': {'large': 'https:

In [71]:
from datetime import datetime, timedelta
from random import randint
import pandas as pd
from dateutil.parser import parse

users, uploads = [], []

In [72]:
for i, result in enumerate(users_json['results']):
    row = {}
    row['UserId'] = i
    row['Title'] = result['name']['title']
    row['FirstName'] = result['name']['first']
    row['LastName'] = result['name']['last']
    row['Email'] = result['email']
    row['Username'] = result['login']['username']
    dob = parse(result['dob']['date'])  
    #print(dob.date())
    row['DOB'] = dob.date()
    
    users.append(row)

In [73]:
for result in uploads_json:
    row = {}
    row['UploadId'] = result['id']
    row['UserId'] = result['userId']
    row['Title'] = result['title']
    row['Body'] = result['body']
    delta = timedelta(seconds=randint(1,86400))
    row['Timestamp'] = datetime.now() - delta
    
    uploads.append(row)

In [74]:
Session = sessionmaker(bind=engine)
session = Session()

In [75]:
for user in users:
    row = Users(**user)
    session.add(row)
    
for upload in uploads:
    row = Uploads(**upload)
    session.add(row)

session.commit()
